<a href="https://colab.research.google.com/github/ayakow1/ttic31220-japanparliament-analysis/blob/main/BERTopic_committee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
!pip install bertopic

In [59]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [60]:
%%capture
!pip install googletrans==3.1.0a0

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sqlite3
import pandas as pd
from bertopic import BERTopic
import numpy as np
from googletrans import Translator

In [4]:
# Import Data
use_raw = True
if use_raw:
    name = "raw_speech"
else:
    name = "speech"

In [5]:
conn = sqlite3.connect(f'/content/drive/MyDrive/議事録/{name}.db')
all = pd.read_sql_query(f'''SELECT * FROM {name} WHERE house ='衆議院' AND speech_date >= '2020-01-01' AND speech_date <= '2023-04-31' ''', conn)
docs = all['speech'].to_list()
conn.close()

In [29]:
topic_model = BERTopic.load("/content/drive/MyDrive/議事録/BERTopic/model_shuugiin_3y")

In [30]:
topic_model.get_topic_info()[:15]

,Topic,Count,Name
0,-1,34792,-1_です_という_こと_ない
1,0,3135,0_ください_でしょ_よろしい_大臣
2,1,1425,1_雇用_賃上げ_賃金_最低賃金
3,2,1386,2_接種_ワクチン_副反応_高齢者
4,3,1006,3_消費税_課税_税制_法人税
5,4,859,4_病院_医療_医師_患者
6,5,725,5_ありがとう_ござい_まし_拍手
7,6,669,6_予算_補正予算_予備費_経費
8,7,641,7_学校_教員_学級_研修
9,8,629,8_原発_原子力_規制_稼働


In [8]:
docs = all['speech'].to_list()
len(docs)

71357

In [9]:
classes = all["committee"].to_list()

In [10]:
# Reduce the number of committees in the dataframe
num_comm = 10
target_comm = all["committee"].value_counts()[:num_comm].index.tolist()

In [11]:
doc_info = topic_model.get_document_info(docs)

In [65]:
allm = pd.concat([all.reset_index(drop=True),doc_info],axis=1,ignore_index=True)
allm.columns=list(all.columns)+list(doc_info.columns)

In [66]:
allm["classes"] = np.where(allm["committee"].isin(target_comm), allm["committee"], "その他")

In [67]:
allm = allm[(allm["Topic"] != -1) & (allm["Topic"] != 0) & (allm["Topic"] != 5)]

In [71]:
trans = Translator()

result = pd.DataFrame(allm.groupby('classes')['Name'].value_counts().groupby(level=0).nlargest(1))
result.columns = ["Count"]
result.index.names = ["0", "Committee", "Topic"]
result = result.reset_index()
result = result.iloc[:,1:4]
result["Topic_t"] = [" ".join([trans.translate(k).text for k in t.split("_")]) for t in result["Topic"]]

In [72]:
result

,Committee,Topic,Count,Topic_t
0,その他,7_学校_教員_学級_研修,478,7 School teacher class training
1,予算委員会,2_接種_ワクチン_副反応_高齢者,314,2 inoculation vaccination side reaction senior...
2,内閣委員会,1_雇用_賃上げ_賃金_最低賃金,167,1 employment wage increase wage minimum wage
3,厚生労働委員会,2_接種_ワクチン_副反応_高齢者,570,2 inoculation vaccination side reaction senior...
4,国土交通委員会,12_住宅_家賃_空き家_マンション,143,12 housing rent empty house apartment
5,外務委員会,9_ウクライナ_ロシア_侵略_侵攻,173,9 Ukraine Russia aggression invasion
6,法務委員会,25_裁判_判決_裁判官_審理,144,25 trial judgment judge trial
7,経済産業委員会,8_原発_原子力_規制_稼働,143,8 nuclear power plant nuclear power regulation...
8,総務委員会,24_nhk_受信料_放送_会長,214,24 nhk reception fee broadcast chairman
9,財務金融委員会,3_消費税_課税_税制_法人税,390,3 sale tax taxation tax system Corporate tax


In [ ]:
result = pd.DataFrame(allm.groupby('classes')['Name'].value_counts().groupby(level=0).nlargest(1))
result.columns = ["Count"]
result.index.names = ["0", "Committee", "Topic"]
result = result.reset_index()
result = result.iloc[:,1:3]

In [ ]:
allmm = pd.concat([allm.reset_index(drop=True),pd.Series(classes).to_frame().T],axis=1,ignore_index=True)
allmm

In [11]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)

11it [00:12,  1.11s/it]


In [16]:
topic_model.visualize_topics_per_class(topics_per_class, topics=[i for i in range(11) if i!=0 and i!=5])